In [ ]:
20

In [ ]:
20

In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model




### Register model

In [2]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [3]:


from federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_with_summation_1MINE_otherDiffLoss_VAE import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


Device: cuda


### Register metric

### Create the config

#### Set data, model, trainer and metric

In [4]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("scripts/B-FHTL_exp_scripts/Graph-DT/fed_dom_sep_MI.yaml")
cfg.data.save_dir = 'test_dir'
# cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']
cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance= 1e2
cfg.params.diff_importance = 1
cfg.params.mine_lr = 0.01
cfg.params.eps=1e-20
cfg.params.p=0.
cfg.params.lam = 0.01
cfg.params.recon_importance = 0.
cfg.params.kld_importance = 0.


#### configure other options

In [5]:
#cfg.use_gpu = True 0.05 [0.02-0.03] 0.04-0.06
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'

cfg.federate.method = \
    f'Laplacian_MINE_VAE_test_no_KLD'

#cfg.federate.local_update_steps = 20000000
cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 2000
cfg.federate.client_num = 16
cfg.early_stop.patience = 3000
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 2.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'


In [6]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(F/home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competitionalse)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [7]:
from yacs.config import CfgNode
client_cfg_file = "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_theirs.yaml"# "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [8]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2023-01-10 20:25:54,818 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [9]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_MI_client import LaplacianDomainSeparationMIClient
from federatedscope.contrib.workers.laplacian_server_dom_sep import LaplacianServerDomSep

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2023-01-10 20:25:54,828 (utils:129)INFO: the current machine is at 127.0.1.1
2023-01-10 20:25:54,829 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2023-01-10 20:25:54,829 (utils:132)INFO: the output dir is exp/test_dir/Laplacian_MINE_VAE_test_no_KLD_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230110202554


In [10]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_1MINE_Other_DiffLoss_VAE_client import LaplacianDomainSeparation1MINEOtherDiffClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianDomainSeparation1MINEOtherDiffClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

rho: 0.0
server params: 
encoder_atom.atom_embedding_list.0.weight


/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
2023-01-10 20:33:23,089 (fed_runner:249)INFO: Server #0 has been set up ... 
2023-01-10 20:33:23,103 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 16
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
 

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
local_gnn.convs.0.nn.linears.0.weight
local_gnn.convs.0.nn.linears.0.bias
loca

2023-01-10 20:33:23,288 (trainer:50)WARNING: Will not use monitor in trainer with class <class 'federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_with_summation_1MINE_otherDiffLoss_VAE.LaplacianDomainSeparation1MINE_Other_Diff_VAETrainer'>
2023-01-10 20:33:23,289 (fed_runner:302)INFO: Client 5 has been set up ... 
2023-01-10 20:33:23,299 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 16
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform:

rho: 0.0
rho: 0.0
rho: 0.0
rho: 0.0
rho: 0.0
rho: 0.0


...

Time for aggregation 1: 46.22793197631836
Time for aggregation 2: 106.7957878112793


2023-01-10 21:06:34,206 (client:410)INFO: {'Role': 'Client #2', 'Round': 2, 'Results_raw': {'test_loss': 65.321632, 'test_avg_loss': 1.020651, 'test_imp_ratio': -3751.511793, 'test_total': 64, 'val_loss': 60.661054, 'val_avg_loss': 0.962874, 'val_imp_ratio': -3533.4861, 'val_total': 63}}
2023-01-10 21:06:34,207 (monitor:513)INFO: current_best=-3533.4861, should_save=True
2023-01-10 21:06:34,208 (client:431)INFO: Client: #2, val_imp_ratio: -3533.4861. model saved at exp/test_dir/Laplacian_MINE_VAE_test_no_KLD_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230110202554/model2.pth
2023-01-10 21:06:34,496 (client:410)INFO: {'Role': 'Client #3', 'Round': 2, 'Results_raw': {'test_loss': 695.010515, 'test_avg_loss': 1.654787, 'test_imp_ratio': -195.97334, 'test_total': 420, 'val_loss': 714.266716, 'val_avg_loss': 1.700635, 'val_imp_ratio': -204.173685, 'val_total': 420}}
2023-01-10 21:06:34,498 (monitor:513)INFO: current_best=-144.129197, should_save=False
2023-01-10 21:06:34,672 (client:410)INFO: {

Time for aggregation 1: 39.162635803222656
Time for aggregation 2: 81.92682266235352


2023-01-10 21:24:13,128 (client:410)INFO: {'Role': 'Client #2', 'Round': 3, 'Results_raw': {'test_loss': 48.265923, 'test_avg_loss': 0.754155, 'test_imp_ratio': -2745.868075, 'test_total': 64, 'val_loss': 44.689132, 'val_avg_loss': 0.709351, 'val_imp_ratio': -2576.797363, 'val_total': 63}}
2023-01-10 21:24:13,129 (monitor:513)INFO: current_best=-2576.797363, should_save=True
2023-01-10 21:24:13,131 (client:431)INFO: Client: #2, val_imp_ratio: -2576.797363. model saved at exp/test_dir/Laplacian_MINE_VAE_test_no_KLD_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230110202554/model2.pth
2023-01-10 21:24:13,410 (client:410)INFO: {'Role': 'Client #3', 'Round': 3, 'Results_raw': {'test_loss': 673.922431, 'test_avg_loss': 1.604577, 'test_imp_ratio': -186.992879, 'test_total': 420, 'val_loss': 687.857094, 'val_avg_loss': 1.637755, 'val_imp_ratio': -192.927032, 'val_total': 420}}
2023-01-10 21:24:13,411 (monitor:513)INFO: current_best=-144.129197, should_save=False
2023-01-10 21:24:13,573 (client:410)

KeyboardInterrupt: 

client 0 times: 0.346s
bei normal fedavg: 0.15s
mit MINE: 1.7 Sekunden


+

In [ ]:
777-431


In [ ]:
data

In [ ]:
import os

,,0After 10 rounds:

diff: 0, csd: 0:
server: -8
c1 t/val: 0.375 /0.37
c2 t/val: 0.678 /0.606
c3 t/val: 0.699 /0.608
c4 t/val: 0.599 /0.537

diff: 0, csd: 0:
server: -4.5
c1 t/val: 0.372879 /0.328
c2 t/val: 0.678992 /0.605809
c3 t/val: 0.70166 /0.608609
c4 t/val: 0.597455 /0.534253



diff: 10, csd: 0:
server: -35
c1 t/val: 0.482/0.52
c2 t/val: 0.694/0.65
c3 t/val: 0.724/0.65
c4 t/val: 0.674/0.575

diff: 1, csd: 0:
server: -9.6
c1 t/val: 0.378194/0.319349
c2 t/val: 0.67461/0.609163
c3 t/val: 0.705559/0.615019
c4 t/val: 0.605581/0.572086

diff: 0, csd: 1e4:
server: -18
c1 t/val: 0.280962/0.23074
c2 t/val: 0.682652/0.603175
c3 t/val: 0.699094/0.67027
c4 t/val: 0.594985/0.609611

diff: 0 csd: 1e8
server: -18
c1 t/val: 0.2606/0.348(0.197)
c2 t/val: 0.653/0.0.602
c3 t/val: 0.691/0.597
c4 t/val: 0.581/0.629

diff: 0 csd: 1e8
server: -12.7
c1 t/val: 0.262647/0.249875
c2 t/val: 0.649273/0.0.600435
c3 t/val: 0.696132/0.615158
c4 t/val: 0.586597/0.656992


diff: 0.1 csd: 0
server: -11
c1 t/val: 0.2606/0.332837
c2 t/val: 0.678326/0.0.610099
c3 t/val: 0.704083/0.616179
c4 t/val: 0.604177/0.559658




In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']